In [ ]:
!pip install beautifulsoup4
!pip install nltk
!pip install requests

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Step 1: Fetch Financial News Headlines from yfinance
def fetch_news(ticker):
    stock = yf.Ticker(ticker)
    news = stock.news
    headlines = []

    for article in news:
        date_time = article['publishedAt']
        title = article['title']
        source = article['source']
        headlines.append((date_time, source, title))

    return headlines

# Step 2: Prepare Data for Analysis
def prepare_data(headlines):
    data = pd.DataFrame(headlines, columns=['Date_Time', 'Source', 'Headline'])
    data['Date_Time'] = pd.to_datetime(data['Date_Time'])
    return data

# Step 3: Sentiment Analysis
def analyze_sentiment(data):
    nltk.download('vader_lexicon')
    sia = SentimentIntensityAnalyzer()

    # Augment VADER with domain-specific terms (example)
    custom_sentiments = {
        'bullish': 2.0,
        'bearish': -2.0,
        'uptrend': 1.5,
        'downtrend': -1.5
    }

    # Add custom sentiments to VADER's lexicon
    for word, sentiment in custom_sentiments.items():
        sia.lexicon[word] = sentiment

    data['Compound'] = data['Headline'].apply(lambda x: sia.polarity_scores(x)['compound'])
    data['Sentiment'] = data['Compound'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))
    return data

# Step 4: Visualize Sentiment Trends
def visualize_sentiment(data):
    # Group by date and sentiment
    sentiment_counts = data.groupby([data['Date_Time'].dt.date, 'Sentiment']).size().unstack(fill_value=0)

    # Plotting
    sentiment_counts.plot(kind='bar', stacked=True, figsize=(12, 6))
    plt.title('Sentiment Distribution of Financial News Headlines')
    plt.xlabel('Date')
    plt.ylabel('Number of Headlines')
    plt.xticks(rotation=45)
    plt.legend(title='Sentiment')
    plt.tight_layout()
    plt.show()

# Main function to run the project
def main():
    ticker = 'AAPL'  # Replace with the desired stock ticker
    headlines = fetch_news(ticker)
    data = prepare_data(headlines)
    data_with_sentiment = analyze_sentiment(data)
    visualize_sentiment(data_with_sentiment)

if __name__ == "__main__":
    main()

KeyError: 'publishedAt'